<a href="https://colab.research.google.com/github/pacificspatial/flateau/blob/main/notebook/sdsc_bootcamp_tokyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLATEAUではじめる空間クエリ
by Hiroo Imaki (PSS) 2023/09/29

## はじめに
このSpatial Data Science の Bootcampでは、空間データサイエンスを始めるにあたり、一番重要なスキルである、空間クエリの第一歩について学びます。そのために、最近進歩が目覚ましい、DuckDBとその拡張機能を利用します。DuckDBは、コマンドラインで利用できるだけでなく、Pythonでも簡単にインストールして利用できます。解析するデータは、国交相が進めるPLATEAUの建物データにしました。建物データは元々3DのCityGML形式でオープンデータとして配布されていますが、今回はそのデータをGeoParquet形式にあらかじめ変換したデータを用意しました。

DuckDBは、2019年に開発が始まって、今年空間解析の機能拡張が追加された空間解析を行う人にとってはとても新しいツールです。
これまでは、PostgreSQLやGoogle BigQueryを使って空間データを解析することが多かったのですが、ローカル環境で大きな空間データを解析できるDuckDBは、空間解析を行う人は使えるようになっていたいツールの一つだと思います。

今回のチュートリアルは、SQLの基礎的な知識がある方で、空間データの解析に興味のある方が対象となっています。SQLについてよくわからないという方は、https://www.w3schools.com/sql/ を初めたくさんのサイトがあるので、そちらを参照してください。また、前提として、PythonがNotebook環境で利用できる方を想定しています。ただし、なんの知識がなくても、このノート自体でチュートリアルが完結しているので、上から順番にセルを実行していただければ、手を動かしながら空間クエリについて学べるようになっています。学習に役立つサイトへのリンクも各所に記載しました。

皆さんが、空間クエリを活用して、さまざまな空間的な課題を解決するのに少しでもお役に立てば幸いです。

## 空間クエリとは
![SQLのイメージ](https://github.com/pacificspatial/flateau/blob/main/notebook/image/sql_screenshot.png?raw=1)

SQLとは、Structured Programming Language (シークゥオル、またはエスキューエルと読むことが多い）の略称で、リレーショナルデータベース(RDBMS)に格納されたデータを管理またデータに対し問い合わせするための言語です。文法は、データベースごとに（例えば、PostgreSQL/PostGIS、Bigqery SQL、Snowflake SQL, DuckDB）多少異なることがありますが、大きな違いはないため、 一度学ぶと、比較的簡単に他のデータベースでもクエリを書くことができます。

SQLを頻繁に使う人は、データエンジニア、データサイエンティスト、データ解析を行う人などです。

SQLを学ぶポイント：いわゆる　"the big 6" コマンドと呼ばれる、SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY が文法の骨格です。SQLを書くときは、SQLコマンドは、できれば大文字で書くのが推奨されていますが、必須ではありません。空間クエリに関わるコマンドは、ST_で始まることが多く、代表的なものとしては、ST_BUFFER, ST_CENTERMEDIAN, ST_POLYGONIZE などがあります。実装されているコマンドの数は、データベースごと異なります。ちなみに、ST_は Spatial & Temporal の略です。

* [PostgreSQL/PostGIS](http://cse.naro.affrc.go.jp/yellow/pgisman/2.2.0/reference.html) 約260（ST関数以外も含む）
* [Google BigQuery](https://cloud.google.com/bigquery/docs/geospatial-data) 66
* [Snowflake](https://docs.snowflake.com/en/sql-reference/functions-geospatial) 66
* [DuckDB](https://duckdb.org/docs/extensions/spatial.html) 44


### 空間クエリを学べるリソース:
CARTOのブログには、空間クエリに関する詳しいチュートリアルがあります
 * [5 maps you didn't know you could create with SQL](https://carto.com/blog/create-maps-dataviz-with-sql)
 * [Ultimate Guide to Spatial Joins & Predicates with SQL](https://carto.com/blog/guide-to-spatial-joins-and-predicates-with-sql)
 * [空間クエリチートシート](https://drive.google.com/file/d/1ncaA_CES-lC4ZqzZ7iC__8kDpED9ieFr/view)

今日のプレゼンターのHelen McKenzie が既存のGISユーザ向けに[空間クエリの始め方についての記事](https://www.helenmakesmaps.com/post/how-to-sql-a-guide-for-gis-users)を書いています。

[University of Washington のサイト](https://uwescience.github.io/SQL-geospatial-tutorial/?utm_content=239999238&utm_medium=social&utm_source=linkedin&hss_channel=lcp-5084329)もチュートリアルや演習を通して空間SQLを学べるようになっています。

以上は、CARTOのSpatial Data Science Bootcampsのハンドブックからの抜粋たもので、一部このハンズオンのために内容を改変し日本語に翻訳しています。

また、以下がこのハンズオンの資料を作ったり勉強したりした時に使ったサイトや動画です。
* [Matt Forrest のYouTube動画](https://www.youtube.com/watch?v=ljzpm3Mrw-I&t=1426s)
* [Mark Litwintschik のブログ](https://tech.marksblogg.com/duckdb-gis-spatial-extension.html)
* [Mother Duck YouTube チャンネル](https://www.youtube.com/@motherduckdb)
* [DuckDB Spatial Github Repo](https://github.com/duckdblabs/duckdb_spatial)
* [DuckDB Spatial 機能拡張](https://duckdb.org/docs/extensions/spatial#spatial-copy-functions)
* [DuckDB Spatial ブログ](https://duckdb.org/2023/04/28/spatial.html)
* [Google BigQuery のGIS機能](https://cloud.google.com/bigquery/docs/geospatial-data)


## PLATEAUと都市のデジタルツインについて

[PLATEAU](https://www.mlit.go.jp/plateau/)とは、国土交通省が主導する、日本全国の3D都市モデルの整備・活用・オープンデータ化プロジェクト。2021年度、全国56、2022年度は新たに71、合計127都市の3D都市モデルデータを[CityGML形式でオープンデータとして公開](https://www.geospatial.jp/ckan/dataset/plateau)

[東京都デジタルツイン実現プロジェクト](https://info.tokyo-digitaltwin.metro.tokyo.lg.jp)は、都市のデジタルツインの実装を目指したプロジェクト。PLATEAUのデータも取り込みつつ、東京都の課題解決と都民のQOL向上を目指す。

### 都市のデジタルツインと空間解析
都市のデジタルツインの一つの要は、空間データを活用し、市民への情報を提供したり、課題を解決すること。そのためには、空間データ解析がとても重要な位置を占める。都市モデルが準拠するCityGML形式のデータは、もともと、さまざまな目的合わせ、変換して利用するためのデータフォーマットであるため、可視化のためには、例えば、3D Tiles形式に変換したり、解析のためには、空間解析に向いたデータ形式に変換して利用する必要がある。

* CityGMLの可視化 ---> 3D Tiles
* CityGMLの解析　 ---> GeoParquet, GeoPackage, Shapefile, etc

ただし、データの変換には、今のところ専門的な知識が必要とされるのが現状。

### CityGMLの変換
なので、

[FME](https://www.safe.com)で[GeoParquet形式とGeoPackage形式に変換しました](https://github.com/pacificspatial/flateau)。FMEを使ったPLATEAUデータ変換の概要は、[このサイトで](https://www.mlit.go.jp/plateau/learning/tpc04-1/)。

今後は変換した成果だけではなく、[FMEのワークスペースもGithubからアクセスできるようにしたい](https://github.com/pacificspatial/flateau/tree/main/workspace)と思っています。



##  ハンズオンの概要と準備
### ソフトウェアとデータ
ハンズオンでは、DuckDBを使って、GeoParquet形式に変換したPLATEAUの都市モデルデータを検索、解析します。
DuckDBを選んだのにはいくつか理由があります。
* インストールが簡単（通常のDBMSのようにサーバ・クライアントの構成ではない）
* 無茶苦茶早い
* 空間解析のための関数が最近実装
* GeoParquetが使える
* ネイティブPOINT_2D, BOX_2Dジオメトリ
* ベクタ化された実行モデルと列指向データ保存形式 ---> データを効率的クエリできる
* OLAP (online analytical processing)　---> 大量のデータに対して高速で多次元分析を行うソフトウェア。複数のリレーショナル・データ・セットからデータを抽出し、それを多次元形式に再編成することで、非常に高速な処理と非常に洞察に富んだ分析を実現
* インプロセスデータベースエンジン ---> クライアントーサーバ構成ではない。従来のサーバ経由のアクセス時間や通信時間のロスを減らせます．しかも，システム全体を統一的に管理でき，高可用性（システム・ダウンを起こさず，連続稼働する機能）を実現しやすい

DuckDBの位置付けに関するまとめは、[DuckDB入門](https://zenn.dev/notrogue/articles/1193d0ab8d8eda) にうまくまとめられていますので、参考にしてください。

GeoParquetを推すのは、いくつか理由があります
* Cloud Native フォーマットで、データサイズがとても小さくなる
* 最近QGISやFMEでも読み書きができるようになった
* Chris Holms、Javier de la Torre、Tim Shuab、Matt Forrest がいいと言っている

ただし、DuckDBは現時点では、以下の点で制限があります（対応予定ではある）。
* GiSTなどの空間インデックスに対応していない
* GEOMETRYタイプの仕様がフル実装されていない（ZやM、サブタイプなど）。GEOGRAPHYタイプに対応していない
* GEOGRAPHYタイプに対応していないので、緯度経度を使った演算（面積計算など）ができない
* PostGISに比べ利用できる関数がまだ少ない
* GeoParquetをテイティブでサポートしていない（[する予定](https://github.com/duckdblabs/duckdb_spatial/issues/27)）

# ハンズオン
### それでは実際にDuckDBを使ってみます。

In [256]:
# 最初にハンズオンで使うライブラリをインストール
# データを地図として見るためのライブラリ
#!pip install leafmap
#!pip install pydeck

In [133]:
# ライブラリのインポート
# Duckdb is already in Colab!
import duckdb
import pandas as pd
import geopandas as gpd
import leafmap
from shapely import wkt

## DuckDBが動くかテスト

In [192]:
# インメモリではなく、ファイルとしてデータを格納したいとき。この例では、test.dbというデータベースを作成する。
# データベース名、拡張子はなんでも良い。拡張子がなくても良い。
# 以下ではコネクションを作成しているが、デフォルトの設定で作成したもの、各種オプションを明示的に指定したもの、
# 外部データベースを指定したもの、インメモリを指定したもの、などの組み合わせで示した。

# デフォルトでの書き方
con = duckdb.connect('test.db')

# オプションで、allow_unsigned_extensionsをtrueにすると、公式ではないH3拡張機能も読み込めるようになる。
#con = duckdb.connect(database="test.db", read_only=False, config={"allow_unsigned_extensions": "true"});

# in-memoryの場合は以下のように書く
#con = duckdb.connect(database=":memory:", read_only=False, config={"allow_unsigned_extensions": "true"});

# 接続を閉じる場合
#con.close()

In [193]:
# インメモリデータベースを使ってクエリが実行できるかテスト
con.sql('SELECT 42')

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘

# DuckDBの基本操作

In [157]:
# リレーションの確認。クエリの結果を変数に格納して、他のクエリで呼び出せる。
r1 = con.sql('SELECT 42 AS i')
con.sql('SELECT i * 2 AS k FROM r1').show()

┌───────┐
│   k   │
│ int32 │
├───────┤
│    84 │
└───────┘



In [158]:
# インメモリにデータベースを作成。このノートを終了したらメモリから消える。
con.sql('DROP TABLE IF EXISTS ducks;')
con.sql("CREATE TABLE ducks AS SELECT 3 AS age, 'mandarin' AS breed;")

# DuckDBのスペシャルクエリ！
con.sql("FROM ducks;")

┌───────┬──────────┐
│  age  │  breed   │
│ int32 │ varchar  │
├───────┼──────────┤
│     3 │ mandarin │
└───────┴──────────┘

In [159]:
# テーブルを準備
con.sql('CREATE OR REPLACE TABLE junk(i INTEGER)')
con.sql('INSERT INTO junk VALUES (42)')
con.sql('INSERT INTO junk VALUES (21)')
# テーブルを表示
con.table('junk').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
│    21 │
└───────┘



In [160]:
# 外部のファイルを読み込む。csv, parquet, json, Pandas DataFrame, Arrow objectに対応
# 読み込みのための関数を使うと以下のようにファイルを読み込める
#con.read_csv('sample_data/mnist_test.csv')
#con.read_parquet('example.parquet')
con.read_json('sample_data/anscombe.json')


┌─────────┬────────┬────────┐
│ Series  │   X    │   Y    │
│ varchar │ double │ double │
├─────────┼────────┼────────┤
│ I       │   10.0 │   8.04 │
│ I       │    8.0 │   6.95 │
│ I       │   13.0 │   7.58 │
│ I       │    9.0 │   8.81 │
│ I       │   11.0 │   8.33 │
│ I       │   14.0 │   9.96 │
│ I       │    6.0 │   7.24 │
│ I       │    4.0 │   4.26 │
│ I       │   12.0 │  10.84 │
│ I       │    7.0 │   4.81 │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ IV      │    8.0 │   5.76 │
│ IV      │    8.0 │   7.71 │
│ IV      │    8.0 │   8.84 │
│ IV      │    8.0 │   8.47 │
│ IV      │    8.0 │   7.04 │
│ IV      │    8.0 │   5.25 │
│ IV      │   19.0 │   12.5 │
│ IV      │    8.0 │   5.56 │
│ IV      │    8.0 │   7.91 │
│ IV      │    8.0 │   6.89 │
├─────────┴────────┴────────┤
│    44 rows (20 shown)     │
└───────────────────────────┘

In [161]:
# 外部のファイルを読み込む。SQL内に直接ファイルパスを書くこともできる
con.sql('SELECT * FROM "sample_data/anscombe.json"')

┌─────────┬────────┬────────┐
│ Series  │   X    │   Y    │
│ varchar │ double │ double │
├─────────┼────────┼────────┤
│ I       │   10.0 │   8.04 │
│ I       │    8.0 │   6.95 │
│ I       │   13.0 │   7.58 │
│ I       │    9.0 │   8.81 │
│ I       │   11.0 │   8.33 │
│ I       │   14.0 │   9.96 │
│ I       │    6.0 │   7.24 │
│ I       │    4.0 │   4.26 │
│ I       │   12.0 │  10.84 │
│ I       │    7.0 │   4.81 │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ IV      │    8.0 │   5.76 │
│ IV      │    8.0 │   7.71 │
│ IV      │    8.0 │   8.84 │
│ IV      │    8.0 │   8.47 │
│ IV      │    8.0 │   7.04 │
│ IV      │    8.0 │   5.25 │
│ IV      │   19.0 │   12.5 │
│ IV      │    8.0 │   5.56 │
│ IV      │    8.0 │   7.91 │
│ IV      │    8.0 │   6.89 │
├─────────┴────────┴────────┤
│    44 rows (20 shown)     │
└───────────────────────────┘

In colab, a multiple line edit/ a mass edit hortcut.
1. Alt + Click for Windows
2. Option + Click for MacOS

And to select all occurrences of the existing selection, use:

1. Ctrl + Shift + L for Windows
2. Command + Shift + L for MacOS

In [162]:
# 読み込んだデータをDataFrameなどに格納もできる
#con.sql('SELECT * FROM "sample_data/anscombe.json"').fetchall()   # Python objects
junk_df = con.sql('SELECT * FROM "sample_data/anscombe.json"').df()         # Pandas DataFrame
#con.sql('SELECT * FROM "sample_data/anscombe.json"').pl()         # Polars DataFrame
#con.sql('SELECT * FROM "sample_data/anscombe.json"').arrow()      # Arrow Table
#con.sql('SELECT * FROM "sample_data/anscombe.json"').fetchnumpy() # NumPy Arrays

junk_df.head()

,Series,X,Y
0,I,10.0,8.04
1,I,8.0,6.95
2,I,13.0,7.58
3,I,9.0,8.81
4,I,11.0,8.33


## Extension を確認

In [163]:
# 拡張機能の確認。デフォルトでは、httpfs, spatialなどは読み込まれていない。
con.sql('select * from duckdb_extensions()')

┌──────────────────┬─────────┬───────────┬──────────────────────┬──────────────────────────────────┬───────────────────┐
│  extension_name  │ loaded  │ installed │     install_path     │           description            │      aliases      │
│     varchar      │ boolean │  boolean  │       varchar        │             varchar              │     varchar[]     │
├──────────────────┼─────────┼───────────┼──────────────────────┼──────────────────────────────────┼───────────────────┤
│ autocomplete     │ false   │ false     │                      │ Add supports for autocomplete …  │ []                │
│ fts              │ true    │ true      │ (BUILT-IN)           │ Adds support for Full-Text Sea…  │ []                │
│ httpfs           │ false   │ true      │ /root/.duckdb/exte…  │ Adds support for reading and w…  │ [http, https, s3] │
│ icu              │ true    │ true      │ (BUILT-IN)           │ Adds support for time zones an…  │ []                │
│ inet             │ false   │ f

## Extensionをインストールして読み込む

In [194]:
# HTTPS経由でファイルを読み込むための機能と、空間データ取扱のための機能を読み込む
con.sql("INSTALL 'httpfs'");
con.sql("INSTALL 'spatial'");

con.sql('LOAD httpfs')
con.sql('LOAD spatial')

## 読み込める空間データファイルフォーマットを確認

In [195]:
# ドライバを確認。なんとGeoParquetはサポートされていない。でもParquetはサポートされているので大丈夫！
# そのうちサポートされるらしい。https://github.com/duckdblabs/duckdb_spatial/issues/27
con.sql('SELECT * FROM st_drivers()').df()

,short_name,long_name,can_create,can_copy,can_open,help_url
0,ESRI Shapefile,ESRI Shapefile,True,False,True,https://gdal.org/drivers/vector/shapefile.html
1,MapInfo File,MapInfo File,True,False,True,https://gdal.org/drivers/vector/mitab.html
2,UK .NTF,UK .NTF,False,False,True,https://gdal.org/drivers/vector/ntf.html
3,LVBAG,Kadaster LV BAG Extract 2.0,False,False,True,https://gdal.org/drivers/vector/lvbag.html
4,S57,IHO S-57 (ENC),True,False,True,https://gdal.org/drivers/vector/s57.html
5,DGN,Microstation DGN,True,False,True,https://gdal.org/drivers/vector/dgn.html
6,OGR_VRT,VRT - Virtual Datasource,False,False,True,https://gdal.org/drivers/vector/vrt.html
7,Memory,Memory,True,False,True,NaN
8,CSV,Comma Separated Value (.csv),True,False,True,https://gdal.org/drivers/vector/csv.html
9,GML,Geography Markup Language (GML),True,False,True,https://gdal.org/drivers/vector/gml.html


# ハンズオンのデータをDuckDBで取り扱う

## Geometryを取り扱えるか確認

In [196]:
con.sql('SELECT ST_POINT(0,0)')

┌────────────────┐
│ st_point(0, 0) │
│    geometry    │
├────────────────┤
│ POINT (0 0)    │
└────────────────┘

## 今回使うデータのリスト
* [建物データについて](https://github.com/pacificspatial/flateau/tree/main/data/plateau)
  * [建物セントロイド](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_centroid.parquet)
  * [建物フットプリントポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_polygon.parquet)
  * [洪水浸水想定：属性テーブル](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_flooding.parquet)
  * [高潮浸水想定：属性テーブル](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_high_tide.parquet)
  * [地滑り：属性テーブル](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_land_slide.parquet)
* [地形データについて](https://github.com/pacificspatial/flateau/blob/main/data/topography.md)
  * [標高 H3 Level10 ポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet)
  * [斜面傾斜 H3 Level10 ポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet)

データを視覚化してみたいときは、WindowsならQGISの最新版でGeoParquetがそのままで表示できます。

Macでは、最新版のQGISインストーラでインストールされるQGISではまだGeoParquetが読めないので、[この記事の手順](https://atsik-ai.com/posts/qgis-geoparquet)に従ってQGISをインストールすると読めます。


## ハンズオン用のデータを読み込む


*   GeoParquetは [ST_READ()](https://duckdb.org/docs/extensions/spatial#spatial-table-functions) ではそのまま読めないので、Parquetとして読む
*   読み込む際に、[ST_GeomFromWKB(geometry)](https://duckdb.org/docs/extensions/spatial#geometry-conversion) で blob で読み込まれるGeometryを変換する
*   Geometryを [DuckDBのネイティブジオメトリタイプ(POINT_2D等)](https://duckdb.org/docs/extensions/spatial.html) にキャストとして読み込むこともできる。ただし、[ネイティブタイプに対応した関数](https://github.com/duckdblabs/duckdb_spatial#supported-functions)はまだ限られている。




In [197]:
# 建物2Dポリゴンの読み込み。時間がかかると思う。httpfs 機能拡張を読み込んだので、インターネット経由でデータが読み込める
polygon_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_polygon.parquet'

con.sql(
    "CREATE OR REPLACE TABLE building_polygon AS SELECT id, gml_id, h3index10, cal_zmin_m, cal_area_m2, \
    cal_height_m::double cal_height_m, ST_GeomFromWKB(geometry) AS geom FROM '{}' ".format(polygon_file)
    )

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [198]:
con.sql('SELECT * FROM building_polygon LIMIT 3')

┌───────┬──────────────────────┬────────────────────┬───┬───────────────────┬───────────────────┬──────────────────────┐
│  id   │        gml_id        │     h3index10      │ … │    cal_area_m2    │   cal_height_m    │         geom         │
│ int64 │       varchar        │       uint64       │   │      double       │      double       │       geometry       │
├───────┼──────────────────────┼────────────────────┼───┼───────────────────┼───────────────────┼──────────────────────┤
│     1 │ bldg_fc50c7d9-76ac…  │ 622329813002289151 │ … │ 40.44564363531246 │ 6.734999999999999 │ POLYGON ((139.7123…  │
│     2 │ bldg_2e11a57e-c315…  │ 622329810539020287 │ … │ 53.06161553075292 │             9.833 │ POLYGON ((139.7071…  │
│     3 │ bldg_72de059a-ddf0…  │ 622329810539118591 │ … │ 64.26319241025253 │             6.605 │ POLYGON ((139.7065…  │
├───────┴──────────────────────┴────────────────────┴───┴───────────────────┴───────────────────┴──────────────────────┤
│ 3 rows                        

In [199]:
# テーブルをDescribe
con.sql('DESCRIBE building_polygon')

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name  │ column_type │  null   │   key   │ default │ extra │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ int32 │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ id           │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ gml_id       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ h3index10    │ UBIGINT     │ YES     │ NULL    │ NULL    │  NULL │
│ cal_zmin_m   │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ cal_area_m2  │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ cal_height_m │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ geom         │ GEOMETRY    │ YES     │ NULL    │ NULL    │  NULL │
└──────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

In [200]:
con.sql('SELECT count(*) FROM building_polygon')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1768280 │
└──────────────┘

In [201]:
con.sql('select cal_zmin_m, cal_area_m2, cal_height_m from building_polygon').df()

,cal_zmin_m,cal_area_m2,cal_height_m
0,2.406,40.445644,6.735
1,2.792,53.061616,9.833
2,2.642,64.263192,6.605
3,2.483,39.170531,8.929
4,2.677,48.882327,6.083
...,...,...,...
1768275,2.666,61.282268,6.539
1768276,2.912,66.133218,7.058
1768277,2.780,50.205177,6.138
1768278,3.000,21.072562,2.236


In [202]:
# 標高データの読み込み（H3 level 10 のインデックスを含む）
elev_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet'
con.sql("CREATE OR REPLACE TABLE elevation AS SELECT h3index10, val_max, val_mean, val_median, val_mode, ST_GeomFromWKB(geometry) AS geom FROM '{}'".format(elev_file))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [203]:
# 斜面傾斜データの読み込み（H3 level 10 のインデックスを含む）
slope_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet'
con.sql("CREATE OR REPLACE TABLE slope AS SELECT h3index10, val_max, val_mean, val_median, val_mode, ST_GeomFromWKB(geometry) AS geom FROM '{}'".format(slope_file))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [204]:
# 東京23区の標高分布
con.sql('SELECT val_max, val_mean, val_median, val_mode FROM elevation').df()

,val_max,val_mean,val_median,val_mode
0,6.19,2.178936,3.040,0.60
1,6.20,2.296221,3.205,3.36
2,6.18,2.424900,3.260,3.41
3,3.55,2.315523,2.250,2.25
4,6.00,3.568281,3.460,3.38
...,...,...,...,...
46727,7.53,3.395628,2.970,2.30
46728,7.60,5.528303,6.885,2.33
46729,12.43,7.627472,7.260,7.29
46730,6.92,2.909865,2.400,2.32


In [205]:
con.sql('SELECT val_max, val_mean, val_median, val_mode FROM slope').df()

,val_max,val_mean,val_median,val_mode
0,2.131054,0.610836,0.549421,0.298201
1,2.484312,0.704333,0.661217,0.654369
2,2.429445,0.844826,0.753571,0.469612
3,1.485541,0.495957,0.468135,0.075518
4,14.944903,1.953333,0.235748,0.111581
...,...,...,...,...
48651,1.653520,0.549164,0.524703,0.231044
48652,4.091684,1.030420,0.892122,0.572825
48653,NaN,NaN,NaN,NaN
48654,1.847704,0.770767,0.750116,0.405783


## 空間クエリ101


### 緯度経度、ジオメトリの取り扱いについて基本的なことを確認
* geometry ST_Point(float x_lon, float y_lat);
* Tokyo Station (latitude:35.6812362, longitude:139.7645499)
* EPSG (European Petroleum Survey Group) コードについて、初めての人は以下を参照してください。
 * https://epsg.io/
 * https://epsg.org/home.html


In [206]:
#緯度、経度 (Google)
con.sql("SELECT ST_POINT(35.6812362,139.7645499) geom")

┌────────────────────────────────┐
│              geom              │
│            geometry            │
├────────────────────────────────┤
│ POINT (35.6812362 139.7645499) │
└────────────────────────────────┘

In [207]:
#経度、緯度 (PostGIS: https://postgis.net/docs/ST_Point.html)
con.sql("SELECT ST_POINT(139.7645499, 35.6812362) geom")

┌────────────────────────────────┐
│              geom              │
│            geometry            │
├────────────────────────────────┤
│ POINT (139.7645499 35.6812362) │
└────────────────────────────────┘

In [208]:
# ST_POINT() では、本来、経度,緯度 の順番のはずなのだが、、、そうするとうまく動かないので、、、、
con.sql("SELECT ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:32654') geom")

┌─────────────────────────────────────────────┐
│                    geom                     │
│                  geometry                   │
├─────────────────────────────────────────────┤
│ POINT (388202.6519479657 3949296.928615559) │
└─────────────────────────────────────────────┘

In [209]:
# 経度・緯度のままだとうまくいかない
con.sql("SELECT ST_TRANSFORM(ST_POINT(139.7645499, 35.6812362), 'EPSG:4326', 'EPSG:32654') geom")

┌───────────────────────────┐
│           geom            │
│         geometry          │
├───────────────────────────┤
│ POINT (Infinity Infinity) │
└───────────────────────────┘

In [210]:
# 経度・緯度をフリップして緯度・経度にしてから計算するとうまくいく
con.sql("SELECT ST_TRANSFORM(ST_FlipCoordinates(ST_POINT(139.7645499,35.6812362)), 'EPSG:4326', 'EPSG:32654') geom")

┌─────────────────────────────────────────────┐
│                    geom                     │
│                  geometry                   │
├─────────────────────────────────────────────┤
│ POINT (388202.6519479657 3949296.928615559) │
└─────────────────────────────────────────────┘

In [211]:
# 東京と仙台の直線距離を計算。単位は、空間参照系の定義で決まります。この場合UTMなのでメートルが単位です。
con.sql("""
SELECT ST_DISTANCE(
  ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:32654'),
  ST_TRANSFORM(ST_POINT(38.3140913,140.4391724), 'EPSG:4326', 'EPSG:32654')
) distance
""")

┌──────────────────┐
│     distance     │
│      double      │
├──────────────────┤
│ 298197.928068306 │
└──────────────────┘

In [212]:
# ちなみに東京と仙台の直線距離を計算。緯度経度を投影しないで計算させると、、、気をつけましょう。
con.sql("""
SELECT ST_DISTANCE(
  ST_POINT(35.6812362,139.7645499),
  ST_POINT(38.3140913,140.4391724)
) distance
""")

┌───────────────────┐
│     distance      │
│      double       │
├───────────────────┤
│ 2.717911237531914 │
└───────────────────┘

In [213]:
# 東京駅からの直線距離を計算. XYを交換した後に計算。。。。
con.sql("select geom, ST_TRANSFORM(ST_FlipCoordinates(geom), 'EPSG:4326', 'EPSG:6677') from building_polygon limit 3")


┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│         geom         │               st_transform(st_flipcoordinates(geom), 'EPSG:4326', 'EPSG:6677')                │
│       geometry       │                                           geometry                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((139.7123…  │ POLYGON ((-50851.20000016458 -10973.879999973837, -50854.140000166124 -10974.53999997561, -…  │
│ POLYGON ((139.7071…  │ POLYGON ((-50972.0100001638 -11441.21999997411, -50980.69500016411 -11445.509999973247, -50…  │
│ POLYGON ((139.7065…  │ POLYGON ((-50877.45000016399 -11500.25999997442, -50883.6750001647 -11504.729999973444, -50…  │
└──────────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘

### H3インデックスを使って空間クエリ
DuckDB には、今の所ジオメトリに対して空間インデックスをPostGISのようには付与できないので、H3インデックスを明示的に使って空間結合する。

In [214]:
# 建物が立っている地形の最大傾斜の大きい順に上位10件の建物のリストを取得
query_01 = '''
  SELECT t1.gml_id, t1.geom, t2.val_median
  FROM building_polygon t1, slope t2
  WHERE t1.h3index10 = t2.h3index10
  ORDER BY val_max DESC
  LIMIT 20
'''
con.sql(query_01)

┌──────────────────────┬──────────────────────────────────────────────────────────────────────────┬────────────────────┐
│        gml_id        │                                   geom                                   │     val_median     │
│       varchar        │                                 geometry                                 │       double       │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────┼────────────────────┤
│ bldg_26811f7d-0a49…  │ POLYGON ((139.74449723500115 35.66740033892029, 139.74445959311024 35.…  │  8.298046588897705 │
│ bldg_3e9e4e15-3683…  │ POLYGON ((139.74477272466495 35.66700668755646, 139.7444687328894 35.6…  │  8.298046588897705 │
│ bldg_660a6fe6-dbd0…  │ POLYGON ((139.74373599923877 35.667795773549145, 139.74374646925907 35…  │  8.298046588897705 │
│ bldg_6d4b9ab2-db75…  │ POLYGON ((139.73544700956978 35.680677084621124, 139.73511680749166 35…  │  2.653118133544922 │
│ bldg_320b7641-aa4e…  │ POLYGON

### 距離を使った空間クエリ

In [215]:
# 東京駅から全ての23区の建物への直線距離を計算
con.sql('''
  SELECT
    t1.gml_id,
    ST_DISTANCE(
      ST_TRANSFORM(ST_FlipCoordinates(geom), 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677')
      ) distance
  FROM building_polygon t1
''')

┌───────────────────────────────────────────┬────────────────────┐
│                  gml_id                   │      distance      │
│                  varchar                  │       double       │
├───────────────────────────────────────────┼────────────────────┤
│ bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928 │ 16199.534429526975 │
│ bldg_2e11a57e-c315-4351-8cca-22649d9763dc │ 16456.578249583657 │
│ bldg_72de059a-ddf0-4959-be00-db8108ca4a3a │ 16383.966347922305 │
│ bldg_2297d74f-b3d5-410a-b62c-a41f6913758c │   16387.7002811633 │
│ bldg_f6edafc1-b722-4b15-90e3-393cbb64d998 │ 16249.887186929942 │
│ bldg_c08f89da-8bf6-42f8-9d80-a45d7fea6a7f │  16430.63891037063 │
│ bldg_de0300aa-4958-41d2-adf7-f65282e67310 │ 16366.930487653508 │
│ bldg_cfeeaa46-c9c1-4e96-813d-8cacf91ad07c │ 16425.888308743528 │
│ bldg_a5f790cc-2b40-4717-a650-0b884dc74123 │ 16341.309520974342 │
│ bldg_1c6f6995-0609-4cd9-b989-7613f97e7829 │ 16468.704803247823 │
│                     ·                     │          ·      

In [216]:
# 東京駅から2km以内の建物の数、合計フットプリント面積、建物の高さの平均値を計算
# この場合、H3インデックスを使えないので、実行速度は遅い。H3ライブラリが使えれば最適化可能か。
%%time
con.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_DISTANCE(
      ST_TRANSFORM(ST_FlipCoordinates(ST_CENTROID(geom)), 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677')
    ) <= 2000
''')

CPU times: user 945 µs, sys: 0 ns, total: 945 µs
Wall time: 958 µs


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬───────────────────┐
│  cnt  │      tot_area      │    avg_height     │
│ int64 │       double       │      double       │
├───────┼────────────────────┼───────────────────┤
│ 16811 │ 3808606.7679503933 │ 19.89897852596522 │
└───────┴────────────────────┴───────────────────┘

In [217]:
# 同じ問題に対する別のアプローチ
con.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_INTERSECTS(
      ST_TRANSFORM(ST_FlipCoordinates(ST_CENTROID(geom)), 'EPSG:4326', 'EPSG:6677'),
      ST_BUFFER(ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677'), 2000)
    )
''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬───────────────────┬───────────────────┐
│  cnt  │     tot_area      │    avg_height     │
│ int64 │      double       │      double       │
├───────┼───────────────────┼───────────────────┤
│ 16688 │ 3785685.110371938 │ 19.92044852588695 │
└───────┴───────────────────┴───────────────────┘

In [228]:
query_02 = '''
  CREATE OR REPLACE TABLE high_slope_buildings AS
  SELECT t1.gml_id, t1.cal_area_m2, t1.cal_height_m, t1.geom, t2.val_max, t2.h3index10
  FROM building_polygon t1, slope t2
  WHERE t1.h3index10 = t2.h3index10
  ORDER BY val_max DESC
  LIMIT 1000
'''
con.sql(query_02)

### 最大傾斜が高く、標高の低いところにある建物のリスト

In [248]:
query_03 = '''
SELECT t1.gml_id, t1.cal_area_m2, t1.cal_height_m, t1.val_max slope_max, t2.val_median elev_median, ST_AsText(t1.geom) geom
FROM high_slope_buildings t1, elevation t2
WHERE t1.h3index10 = t2.h3index10
ORDER BY elev_median ASC
LIMIT 100
'''
con.sql(query_03)

┌──────────────────────┬────────────────────┬────────────────────┬───┬───────────────────┬──────────────────────┐
│        gml_id        │    cal_area_m2     │    cal_height_m    │ … │    elev_median    │         geom         │
│       varchar        │       double       │       double       │   │      double       │       varchar        │
├──────────────────────┼────────────────────┼────────────────────┼───┼───────────────────┼──────────────────────┤
│ bldg_5b850222-9a3a…  │  45.22805555742468 │              8.924 │ … │               3.0 │ POLYGON ((139.7473…  │
│ bldg_c3cd8d33-abc9…  │ 48.845026593104706 │  8.743000000000002 │ … │               3.0 │ POLYGON ((139.7472…  │
│ bldg_8691f828-1f12…  │   55.4216705138908 │  6.841999999999999 │ … │               3.0 │ POLYGON ((139.7475…  │
│ bldg_df440e63-a3ca…  │ 138.98993825633923 │ 6.7170000000000005 │ … │               3.0 │ POLYGON ((139.7477…  │
│ bldg_05ea581b-49df…  │  53.37232018602088 │              9.016 │ … │               3.0

### DuckDBのネイティブジオメトリタイプを試して見る

In [232]:
# DuckDBのネイティブ POINT_2D ジオメトリを追加してみる
con.sql('ALTER TABLE building_polygon ADD IF NOT EXISTS geom_2dp POINT_2D')
con.sql('update building_polygon set geom_2dp = ST_CENTROID(ST_FlipCoordinates(geom))::POINT_2D')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [233]:
# 東京駅から全ての23区の建物への直線距離を計算
%%time
con.sql('''
  SELECT
    t1.gml_id,
    ST_DISTANCE(
      ST_TRANSFORM(geom_2dp, 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499)::POINT_2D, 'EPSG:4326', 'EPSG:6677')
      ) distance
  FROM building_polygon t1
''')

CPU times: user 1.14 ms, sys: 0 ns, total: 1.14 ms
Wall time: 1.15 ms


┌───────────────────────────────────────────┬────────────────────┐
│                  gml_id                   │      distance      │
│                  varchar                  │       double       │
├───────────────────────────────────────────┼────────────────────┤
│ bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928 │ 16202.130320538663 │
│ bldg_2e11a57e-c315-4351-8cca-22649d9763dc │  16461.84353808086 │
│ bldg_72de059a-ddf0-4959-be00-db8108ca4a3a │ 16388.850356952862 │
│ bldg_2297d74f-b3d5-410a-b62c-a41f6913758c │ 16391.534073012266 │
│ bldg_f6edafc1-b722-4b15-90e3-393cbb64d998 │ 16252.036103992812 │
│ bldg_c08f89da-8bf6-42f8-9d80-a45d7fea6a7f │ 16435.139746734392 │
│ bldg_de0300aa-4958-41d2-adf7-f65282e67310 │ 16370.600505466939 │
│ bldg_cfeeaa46-c9c1-4e96-813d-8cacf91ad07c │  16428.65350213206 │
│ bldg_a5f790cc-2b40-4717-a650-0b884dc74123 │ 16347.421510492963 │
│ bldg_1c6f6995-0609-4cd9-b989-7613f97e7829 │ 16478.707013098723 │
│                     ·                     │          ·      

In [234]:
# 東京駅から2km以内の建物の数、合計フットプリント面積、建物の高さの平均値を計算
%%time
con.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_DISTANCE(
      ST_TRANSFORM(geom_2dp, 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499)::POINT_2D, 'EPSG:4326', 'EPSG:6677')
    ) < 2000
''')

CPU times: user 1.61 ms, sys: 0 ns, total: 1.61 ms
Wall time: 2.39 ms


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬───────────────────┐
│  cnt  │      tot_area      │    avg_height     │
│ int64 │       double       │      double       │
├───────┼────────────────────┼───────────────────┤
│ 16811 │ 3808606.7679503933 │ 19.89897852596522 │
└───────┴────────────────────┴───────────────────┘

## クエリの結果をGeoJSONで出力
[COPYコマンド](https://duckdb.org/docs/extensions/spatial#spatial-copy-functions)を使うと、クエリの結果をファイルとして出力できる。

In [227]:
# GeoJSONとしてクエリの結果を保存
con.sql("copy ({}) to 'test.geojson' with (format gdal, driver 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES')".format(query_01))


## クエリの結果をDataFrameに格納

In [249]:
# change to DF
df = con.sql(query_03).df()
df

,gml_id,cal_area_m2,cal_height_m,slope_max,elev_median,geom
0,bldg_5b850222-9a3a-4be8-9975-44094f22c779,45.228056,8.924,42.938213,3.000,POLYGON ((139.74736930912675 35.75701553916105...
1,bldg_c3cd8d33-abc9-4a78-8c5e-01b52c6cff87,48.845027,8.743,42.938213,3.000,POLYGON ((139.74727110389452 35.75696669544266...
2,bldg_8691f828-1f12-4b96-b877-13a5d286cb3e,55.421671,6.842,42.938213,3.000,POLYGON ((139.74753260455907 35.75730209807344...
3,bldg_df440e63-a3ca-41e5-969b-72ae2608e052,138.989938,6.717,42.938213,3.000,"POLYGON ((139.747790346849 35.75722287181339, ..."
4,bldg_05ea581b-49df-43ab-8e80-ff88ddc0cacd,53.372320,9.016,42.938213,3.000,POLYGON ((139.74725056456438 35.75704248513314...
...,...,...,...,...,...,...
95,bldg_23f10c73-cc90-4d84-b935-484fba115911,97.847262,33.682,44.739048,6.945,POLYGON ((139.73704538802758 35.65728069509238...
96,bldg_6bfa16f2-b150-46c2-843c-a94d59df24c7,240.015155,19.120,44.739048,6.945,POLYGON ((139.73743150165146 35.65743448210492...
97,bldg_f6271b22-7fcf-4005-ba2d-35fe5f699f55,93.105383,21.112,44.739048,6.945,POLYGON ((139.73714582060416 35.65727623512281...
98,bldg_bb6d28f6-1dc4-4894-b78d-01c06d2a2f7a,40.641346,14.389,44.739048,6.945,POLYGON ((139.73704324148062 35.65716987292308...


## クエリの結果をGeoDataFrameに格納

In [250]:
# change to GDF
df["geom"] = gpd.GeoSeries.from_wkt(df["geom"])
gdf = gpd.GeoDataFrame(df, geometry="geom")
gdf

,gml_id,cal_area_m2,cal_height_m,slope_max,elev_median,geom
0,bldg_5b850222-9a3a-4be8-9975-44094f22c779,45.228056,8.924,42.938213,3.000,"POLYGON ((139.74737 35.75702, 139.74737 35.756..."
1,bldg_c3cd8d33-abc9-4a78-8c5e-01b52c6cff87,48.845027,8.743,42.938213,3.000,"POLYGON ((139.74727 35.75697, 139.74726 35.756..."
2,bldg_8691f828-1f12-4b96-b877-13a5d286cb3e,55.421671,6.842,42.938213,3.000,"POLYGON ((139.74753 35.75730, 139.74746 35.757..."
3,bldg_df440e63-a3ca-41e5-969b-72ae2608e052,138.989938,6.717,42.938213,3.000,"POLYGON ((139.74779 35.75722, 139.74766 35.757..."
4,bldg_05ea581b-49df-43ab-8e80-ff88ddc0cacd,53.372320,9.016,42.938213,3.000,"POLYGON ((139.74725 35.75704, 139.74725 35.756..."
...,...,...,...,...,...,...
95,bldg_23f10c73-cc90-4d84-b935-484fba115911,97.847262,33.682,44.739048,6.945,"POLYGON ((139.73705 35.65728, 139.73699 35.657..."
96,bldg_6bfa16f2-b150-46c2-843c-a94d59df24c7,240.015155,19.120,44.739048,6.945,"POLYGON ((139.73743 35.65743, 139.73744 35.657..."
97,bldg_f6271b22-7fcf-4005-ba2d-35fe5f699f55,93.105383,21.112,44.739048,6.945,"POLYGON ((139.73715 35.65728, 139.73721 35.657..."
98,bldg_bb6d28f6-1dc4-4894-b78d-01c06d2a2f7a,40.641346,14.389,44.739048,6.945,"POLYGON ((139.73704 35.65717, 139.73702 35.657..."


## 地図表示

### leafmapを利用して、データを2D地図表示

In [255]:
# visualize data with 2D leafmap
m = leafmap.Map(center=(35.5744826,139.7078659), zoom=13)
m.add_gdf(gdf, layer_name="Plateau Buildings")
m

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 35.5744826,
    "longitude": 139.7078659,
    "zoom": 13
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": [
        {
          "cal_area_m2": 45.22805555742468,
          "cal_height_m": 8.924,
          "elev_median": 3.0,
          "geometry": {
            "coordinates": [
              [
                [
                  139.74736930912675,
                  35.75701553916105
                ],
                [
                  139.74736684857794,
                  35.75689739973473
                ],
                [
                  139.74732870825954,
                  35.756897927709666
                ],
                [
                  139.74733116875197,
                  35.757016067136675
                ],
                [
                  139.74736930912675,
                  35.75701553916105
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_5b850222-9a3a-4be8-9975-44094f22c779",
          "slope_max": 42.93821334838867
        },
        {
          "cal_area_m2": 48.845026593104706,
          "cal_height_m": 8.743000000000002,
          "elev_median": 3.0,
          "geometry": {
            "coordinates": [
              [
                [
                  139.74727110389452,
                  35.75696669544266
                ],
                [
                  139.74726489289372,
                  35.75688007028916
                ],
                [
                  139.74720606028825,
                  35.75688282239681
                ],
                [
                  139.74721183459036,
                  35.756963137719666
                ],
                [
                  139.7472504308736,
                  35.75696127248968
                ],
                [
                  139.74725086630886,
                  35.75696767245824
                ],
                [
                  139.74727110389452,
                  35.75696669544266
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_c3cd8d33-abc9-4a78-8c5e-01b52c6cff87",
          "slope_max": 42.93821334838867
        },
        {
          "cal_area_m2": 55.4216705138908,
          "cal_height_m": 6.841999999999999,
          "elev_median": 3.0,
          "geometry": {
            "coordinates": [
              [
                [
                  139.74753260455907,
                  35.757302098073446
                ],
                [
                  139.7474593990218,
                  35.75730195569968
                ],
                [
                  139.74745937103208,
                  35.757328004986164
                ],
                [
                  139.74744587995917,
                  35.75732799535438
                ],
                [
                  139.74744585219906,
                  35.757354855864996
                ],
                [
                  139.74745945385905,
                  35.757354865575735
                ],
                [
                  139.74745944184963,
                  35.75736604243211
                ],
                [
                  139.74749029439982,
                  35.757366064453215
                ],
                [
                  139.74749028229786,
                  35.75737733144555
                ],
                [
                  139.7475324141843,
                  35.75737745164056
                ],
                [
                  139.74753260455907,
                  35.757302098073446
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_8691f828-1f12-4b96-b877-13a5d286cb3e",
          "slope_max": 42.93821334838867
        },
        {
          "cal_area_m2": 138.98993825633923,
          "cal_height_m": 6.7170000000000005,
          "elev_

### leafmap/deck.gl を利用して、データを3D地図表示

In [253]:
# visualize data in 3D with deck.gl
import leafmap.deck as leafmap

initial_view_state = {
    "latitude": 35.5744826,
    "longitude": 139.7078659,
    "zoom": 13,
    "pitch": 45,
    "bearing": 10,
}

m = leafmap.Map(initial_view_state=initial_view_state)

m.add_gdf(
    gdf,
    extruded=True,
    get_elevation="cal_height_m",
    elevation_range=[0, 3000],
    elevation_scale=3,
)
m

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "bearing": 10,
    "latitude": 35.5744826,
    "longitude": 139.7078659,
    "pitch": 45,
    "zoom": 13
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": [
        {
          "cal_area_m2": 45.22805555742468,
          "cal_height_m": 8.924,
          "elev_median": 3.0,
          "geometry": {
            "coordinates": [
              [
                [
                  139.74736930912675,
                  35.75701553916105
                ],
                [
                  139.74736684857794,
                  35.75689739973473
                ],
                [
                  139.74732870825954,
                  35.756897927709666
                ],
                [
                  139.74733116875197,
                  35.757016067136675
                ],
                [
                  139.74736930912675,
                  35.75701553916105
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_5b850222-9a3a-4be8-9975-44094f22c779",
          "slope_max": 42.93821334838867
        },
        {
          "cal_area_m2": 48.845026593104706,
          "cal_height_m": 8.743000000000002,
          "elev_median": 3.0,
          "geometry": {
            "coordinates": [
              [
                [
                  139.74727110389452,
                  35.75696669544266
                ],
                [
                  139.74726489289372,
                  35.75688007028916
                ],
                [
                  139.74720606028825,
                  35.75688282239681
                ],
                [
                  139.74721183459036,
                  35.756963137719666
                ],
                [
                  139.7472504308736,
                  35.75696127248968
                ],
                [
                  139.74725086630886,
                  35.75696767245824
                ],
                [
                  139.74727110389452,
                  35.75696669544266
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_c3cd8d33-abc9-4a78-8c5e-01b52c6cff87",
          "slope_max": 42.93821334838867
        },
        {
          "cal_area_m2": 55.4216705138908,
          "cal_height_m": 6.841999999999999,
          "elev_median": 3.0,
          "geometry": {
            "coordinates": [
              [
                [
                  139.74753260455907,
                  35.757302098073446
                ],
                [
                  139.7474593990218,
                  35.75730195569968
                ],
                [
                  139.74745937103208,
                  35.757328004986164
                ],
                [
                  139.74744587995917,
                  35.75732799535438
                ],
                [
                  139.74744585219906,
                  35.757354855864996
                ],
                [
                  139.74745945385905,
                  35.757354865575735
                ],
                [
                  139.74745944184963,
                  35.75736604243211
                ],
                [
                  139.74749029439982,
                  35.757366064453215
                ],
                [
                  139.74749028229786,
                  35.75737733144555
                ],
                [
                  139.7475324141843,
                  35.75737745164056
                ],
                [
                  139.74753260455907,
                  35.757302098073446
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_8691f828-1f12-4b96-b877-13a5d286cb3e",
          "slope_max": 42.93821334838867
        },
        {
          "cal_area_m2": 138.98993825633923,
          "cal_height_m": 